# Testing notebook

In [1]:
from src.seq2seq.dataset import SeqDataset, pad_batch
from functools import partial

pad_batch_with_fixed_length = partial(pad_batch, fixed_length=128)
dataset_path = '/home/gkulemeyer/Documents/Repos/AEseq2seq/data/ArchiveII-KFold/common/fold_0_test.csv'
data = SeqDataset( dataset_path, min_len=0, max_len=512, verbose=False, cache_path=None, for_prediction=False,  training=False)

In [13]:
print([f'{i},{len(data) / i}' for i in range(80,128)]) 

['80,5.8', '81,5.728395061728395', '82,5.658536585365853', '83,5.590361445783133', '84,5.523809523809524', '85,5.458823529411765', '86,5.395348837209302', '87,5.333333333333333', '88,5.2727272727272725', '89,5.213483146067416', '90,5.155555555555556', '91,5.0989010989010985', '92,5.043478260869565', '93,4.989247311827957', '94,4.9361702127659575', '95,4.88421052631579', '96,4.833333333333333', '97,4.783505154639175', '98,4.73469387755102', '99,4.686868686868687', '100,4.64', '101,4.594059405940594', '102,4.549019607843137', '103,4.504854368932039', '104,4.461538461538462', '105,4.419047619047619', '106,4.377358490566038', '107,4.336448598130841', '108,4.296296296296297', '109,4.256880733944954', '110,4.218181818181818', '111,4.18018018018018', '112,4.142857142857143', '113,4.106194690265487', '114,4.0701754385964914', '115,4.034782608695652', '116,4.0', '117,3.965811965811966', '118,3.9322033898305087', '119,3.899159663865546', '120,3.8666666666666667', '121,3.834710743801653', '122,3.

In [3]:
for n in range(3):
    print(n)
    print(data[n]['length'])
    print(data[n]['embedding'].shape)

0
119
torch.Size([4, 119])
1
120
torch.Size([4, 120])
2
120
torch.Size([4, 120])


In [4]:
from torch.utils.data import DataLoader

loader = DataLoader(data, batch_size=1, shuffle=False,collate_fn=pad_batch_with_fixed_length)

next(iter(loader))['embedding'].shape

torch.Size([1, 4, 128])

In [5]:
import torch as tr
x = tr.tensor([[
    [1,0,0,0,1,0,0,0],
    [0,1,0,0,0,1,0,0],
    [0,0,1,0,0,0,1,0],
    [0,0,0,1,0,0,0,1],]
])
x.shape

torch.Size([1, 4, 8])

In [8]:
import pandas as pd
from torch.utils.data import Dataset
import torch as tr
import os
import json
import pickle
import random
from src.seq2seq.embeddings import OneHotEmbedding


class SeqDataset2(Dataset):
    def __init__(
        self,
        dataset_path,
        min_len=0,
        max_len=512,
        verbose=False,
        cache_path=None,
        for_prediction=False,
        training=False,
        n_swaps=0,
        **kargs,
    ):
        """
        interaction_prior: none, probmat
        """
        self.max_len = max_len
        self.verbose = verbose
        if cache_path is not None and not os.path.isdir(cache_path):
            os.mkdir(cache_path)
        self.cache = cache_path

        # Loading dataset
        data = pd.read_csv(dataset_path)
        self.training = training

        assert (
            "sequence" in data.columns and "id" in data.columns
        ), "Dataset should contain 'id' and 'sequence' columns"

        data["len"] = data.sequence.str.len()

        if max_len is None:
            max_len = max(data.len)
        self.max_len = max_len

        datalen = len(data)

        data = data[(data.len >= min_len) & (data.len <= max_len)]

        if len(data) < datalen:
            print(
                f"From {datalen} sequences, filtering {min_len} < len < {max_len} we have {len(data)} sequences"
            )

        self.sequences = data.sequence.tolist()
        self.ids = data.id.tolist()
        self.embedding = OneHotEmbedding()
        self.embedding_size = self.embedding.emb_size
        self.n_swaps = n_swaps

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        seqid = self.ids[idx]
        cache = f"{self.cache}/{seqid}.pk"
        if (self.cache is not None) and os.path.isfile(cache):
            item = pickle.load(open(cache, "rb"))
        else:
            sequence = self.sequences[idx]
            L = len(sequence)
            seq_emb = self.embedding.seq2emb(sequence)
            embedding_with_noise = add_noise(seq_emb, self.n_swaps)

            item = {
                "id": seqid,
                "length": L,
                "sequence": sequence,
                "embedding": seq_emb,
                "embedding_with_noise": embedding_with_noise,
            }

            if self.cache is not None:
                pickle.dump(item, open(cache, "wb"))

        return item


def pad_batch(batch, fixed_length=0):
    """batch is a dictionary with different variables lists"""
    L = [b["length"] for b in batch]
    if fixed_length == 0:
        fixed_length = max(L)
    embedding_pad = tr.zeros((len(batch), batch[0]["embedding"].shape[0], fixed_length))
    embedding_pad_w_noise = tr.zeros(
        (len(batch), batch[0]["embedding_with_noise"].shape[0], fixed_length)
    )
    mask = tr.zeros((len(batch), fixed_length), dtype=tr.bool)

    for k in range(len(batch)):
        embedding_pad[k, :, : L[k]] = batch[k]["embedding"]
        embedding_pad_w_noise[k, :, : L[k]] = batch[k]["embedding_with_noise"]
        mask[k, : L[k]] = 1

    out_batch = {
        "id": [b["id"] for b in batch],
        "length": L,
        "sequence": [b["sequence"] for b in batch],
        "embedding": embedding_pad,
        "embedding_with_noise": embedding_pad_w_noise,
        "mask": mask,
    }

    return out_batch


def add_noise(x, N=0):
    assert N < x.shape[-1], "N should be lower than the shape of x (starting on 0)"

    if N == 0:
        return x

    x_l = [_ for _ in range(x.shape[-1])]
    random.shuffle(x_l)
    v = [0, 1, 2, 3]

    for _ in range(N):
        pos = x_l[-1]
        x_l.pop()
        random.shuffle(v)
        nt = tr.zeros([4], dtype=tr.float)
        nt[v[0]] = 1.0
        x[:, pos] = nt
    return x


In [42]:
pad_batch_with_fixed_length = partial(pad_batch, fixed_length=128)
dataset_path = '/home/gkulemeyer/Documents/Repos/AEseq2seq/data/ArchiveII-KFold/common/fold_0_train.csv'
data2 = SeqDataset2( dataset_path, min_len=0, max_len=512, verbose=False, cache_path=None, for_prediction=False,  training=False, n_swaps=5)

In [43]:
print(data2[0]['embedding_with_noise'].shape)

loader = DataLoader(data2, batch_size=1, shuffle=False,collate_fn=pad_batch_with_fixed_length) 
print(next(iter(loader))['embedding_with_noise'].shape)

print(data2[0]['embedding'].shape)

loader = DataLoader(data2, batch_size=1, shuffle=False,collate_fn=pad_batch_with_fixed_length)
print(next(iter(loader))['embedding'].shape)

torch.Size([4, 119])
torch.Size([1, 4, 128])
torch.Size([4, 119])
torch.Size([1, 4, 128])


In [46]:
abs(next(iter(loader))['embedding_with_noise'].view(-1) - next(iter(loader))['embedding'].view(-1)).sum()


tensor(10.)